In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

Load dan menggabungkan semua dataset menjadi satu

In [2]:
files = [f for f in os.listdir() if len(f.split('.')) > 1 and f.split('.')[1] == 'csv']
files

['Sales_February_2019.csv',
 'Sales_April_2019.csv',
 'Sales_March_2019.csv',
 'Sales_September_2019.csv',
 'Sales_October_2019.csv',
 'Sales_July_2019.csv',
 'Sales_December_2019.csv',
 'Sales_November_2019.csv',
 'Sales_January_2019.csv',
 'Sales_August_2019.csv',
 'Sales_June_2019.csv',
 'Sales_May_2019.csv']

In [3]:
all_dataset = pd.DataFrame()
for f in files:
  dataset = pd.read_csv(f)
  all_dataset = pd.concat([all_dataset, dataset])

all_dataset.to_csv("all_dataset.csv", index=False)

In [4]:
dataset = pd.read_csv("all_dataset.csv")
print(dataset.shape)
dataset.head()

(186850, 6)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,150502,iPhone,1,700,02/18/19 01:35,"866 Spruce St, Portland, ME 04101"
1,150503,AA Batteries (4-pack),1,3.84,02/13/19 07:24,"18 13th St, San Francisco, CA 94016"
2,150504,27in 4K Gaming Monitor,1,389.99,02/18/19 09:46,"52 6th St, New York City, NY 10001"
3,150505,Lightning Charging Cable,1,14.95,02/02/19 16:47,"129 Cherry St, Atlanta, GA 30301"
4,150506,AA Batteries (4-pack),2,3.84,02/28/19 20:32,"548 Lincoln St, Seattle, WA 98101"


In [5]:
dataset = dataset.dropna(how='all')
dataset.shape

(186305, 6)

In [6]:
size = dataset.shape[0]
dataset = dataset[:int(size/10)]

Membuat kolom ID berdasarkan alamat

In [7]:
addresses = dataset['Purchase Address'].unique()
addresses.shape

(17410,)

Membuat dictionary dengan Purchase Address sebagai key dimana valuenya akan mulai dari 1 - jumlah alamat

In [8]:
addresses_dict = dict()
for i, item in enumerate(addresses):
  addresses_dict[item] = i+1

In [9]:
dataset['User ID'] = dataset['Purchase Address'].map(addresses_dict)
dataset.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,User ID
0,150502,iPhone,1,700,02/18/19 01:35,"866 Spruce St, Portland, ME 04101",1
1,150503,AA Batteries (4-pack),1,3.84,02/13/19 07:24,"18 13th St, San Francisco, CA 94016",2
2,150504,27in 4K Gaming Monitor,1,389.99,02/18/19 09:46,"52 6th St, New York City, NY 10001",3
3,150505,Lightning Charging Cable,1,14.95,02/02/19 16:47,"129 Cherry St, Atlanta, GA 30301",4
4,150506,AA Batteries (4-pack),2,3.84,02/28/19 20:32,"548 Lincoln St, Seattle, WA 98101",5


In [10]:
dataset = dataset.loc[(dataset["Quantity Ordered"].str.isnumeric())]

In [11]:
dataset['Quantity Ordered'] = pd.to_numeric(dataset['Quantity Ordered'])
dataset['Price Each'] = pd.to_numeric(dataset['Price Each'])

In [12]:
customer_item_matrix = dataset.pivot_table(
    index='User ID', 
    columns='Product', 
    values='Quantity Ordered',
    aggfunc='sum'
)
customer_item_matrix

Product,20in Monitor,27in 4K Gaming Monitor,27in FHD Monitor,34in Ultrawide Monitor,AA Batteries (4-pack),AAA Batteries (4-pack),Apple Airpods Headphones,Bose SoundSport Headphones,Flatscreen TV,Google Phone,LG Dryer,LG Washing Machine,Lightning Charging Cable,Macbook Pro Laptop,ThinkPad Laptop,USB-C Charging Cable,Vareebadd Phone,Wired Headphones,iPhone
User ID,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17407,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17408,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix

Product,20in Monitor,27in 4K Gaming Monitor,27in FHD Monitor,34in Ultrawide Monitor,AA Batteries (4-pack),AAA Batteries (4-pack),Apple Airpods Headphones,Bose SoundSport Headphones,Flatscreen TV,Google Phone,LG Dryer,LG Washing Machine,Lightning Charging Cable,Macbook Pro Laptop,ThinkPad Laptop,USB-C Charging Cable,Vareebadd Phone,Wired Headphones,iPhone
User ID,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17406,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
17407,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
17408,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
dataset['User ID'].value_counts()

10210    5
11229    4
12852    4
1750     3
6514     3
        ..
6077     1
6078     1
6079     1
6080     1
17410    1
Name: User ID, Length: 17409, dtype: int64

In [15]:
dataset[dataset['User ID'] == 10210]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,User ID
10836,160873,iPhone,1,700.00,02/10/19 10:24,"702 River St, San Francisco, CA 94016",10210
10837,160873,Lightning Charging Cable,1,14.95,02/10/19 10:24,"702 River St, San Francisco, CA 94016",10210
10838,160873,Apple Airpods Headphones,1,150.00,02/10/19 10:24,"702 River St, San Francisco, CA 94016",10210
10839,160873,Wired Headphones,1,11.99,02/10/19 10:24,"702 River St, San Francisco, CA 94016",10210
10840,160873,Google Phone,1,600.00,02/10/19 10:24,"702 River St, San Francisco, CA 94016",10210


In [16]:
customer_item_matrix.loc[10210]

Product
20in Monitor                  0
27in 4K Gaming Monitor        0
27in FHD Monitor              0
34in Ultrawide Monitor        0
AA Batteries (4-pack)         0
AAA Batteries (4-pack)        0
Apple Airpods Headphones      1
Bose SoundSport Headphones    0
Flatscreen TV                 0
Google Phone                  1
LG Dryer                      0
LG Washing Machine            0
Lightning Charging Cable      1
Macbook Pro Laptop            0
ThinkPad Laptop               0
USB-C Charging Cable          0
Vareebadd Phone               0
Wired Headphones              1
iPhone                        1
Name: 10210, dtype: int64

In [17]:
user_user_sim_matrix = pd.DataFrame(
    cosine_similarity(customer_item_matrix)
)
user_user_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,17399,17400,17401,17402,17403,17404,17405,17406,17407,17408
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.000000,0.000000,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
17405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
17406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
17407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
user_user_sim_matrix.columns = customer_item_matrix.index
user_user_sim_matrix['CustomerID'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')
user_user_sim_matrix

User ID,1,2,3,4,5,6,7,8,9,10,...,17401,17402,17403,17404,17405,17406,17407,17408,17409,17410
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.000000,0.000000,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
17407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
17408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Menampilkan User yang paling similiar dengan User ID = 6514

In [26]:
user_user_sim_matrix.loc[6514].sort_values(ascending=False).head(20)

User ID
6514     1.000000
6781     1.000000
4542     1.000000
1982     0.816497
3031     0.816497
5518     0.816497
15751    0.816497
6779     0.816497
4528     0.816497
9144     0.816497
9621     0.816497
7750     0.816497
3739     0.816497
11268    0.816497
15306    0.816497
5174     0.816497
1954     0.816497
14137    0.816497
5696     0.816497
7997     0.816497
Name: 6514, dtype: float64

In [21]:
items_bought_by_A = set(customer_item_matrix.loc[6514].iloc[
    customer_item_matrix.loc[6514].to_numpy().nonzero()
].index)
items_bought_by_A

{'USB-C Charging Cable', 'Vareebadd Phone', 'Wired Headphones'}

Saya tidak memilih User ID 6781 dan 4542 karena kedua User ID ini mempunyai nilai cosine similiarity 1 yang artinya item yang dibeli oleh mereka sama persis dengan item yang dibeli oleh User ID 6514

In [27]:
items_bought_by_B = set(customer_item_matrix.loc[1982].iloc[
    customer_item_matrix.loc[1982].to_numpy().nonzero()
].index)
items_bought_by_B

{'USB-C Charging Cable', 'Wired Headphones'}

In [28]:
items_to_recommend_to_B = items_bought_by_A - items_bought_by_B
dataset.loc[
    dataset['Product'].isin(items_to_recommend_to_B), 
    ['Product']
].drop_duplicates().set_index('Product')

""
Product
Vareebadd Phone


Dari hasil di atas kita dapat menyimpulkan bahwa item yang akan direkomendasikan kepada User B (User dengan ID 1982) adalah Vareebadd Phone. <br>
Hasil rekomendasi yang didapatkan hanya 1 karena jumlah variasi product dari dataset yang terbatas dan juga similiarity yang tinggi antar User